In [ ]:
import time
import random
from seal import EncryptionParameters, scheme_type, \
    SEALContext, print_parameters, KeyGenerator, \
    Encryptor, CoeffModulus, Evaluator, Decryptor, \
    Plaintext, Ciphertext, IntegerEncoder, PlainModulus, \
    BatchEncoder, CKKSEncoder, Int64Vector, UInt64Vector, \
    IntVector, DoubleVector
from numpy import log2

In `example_1_bfv_basics.py' we showed how to perform a very simple computation using the<br>
BFV scheme. The computation was performed modulo the plain_modulus parameter, and<br>
utilized only one coefficient from a BFV plaintext polynomial. This approach has<br>
two notable problems:<br>
<br>
   (1) Practical applications typically use integer or real number arithmetic,<br>
       not modular arithmetic;<br>
   (2) We used only one coefficient of the plaintext polynomial. This is really<br>
       wasteful, as the plaintext polynomial is large and will in any case be<br>
       encrypted in its entirety.<br>
<br>
For (1), one may ask why not just increase the plain_modulus parameter until no<br>
overflow occurs, and the computations behave as in integer arithmetic. The problem<br>
is that increasing plain_modulus increases noise budget consumption, and decreases<br>
the initial noise budget too.<br>
<br>
In these examples we will discuss other ways of laying out data into plaintext<br>
elements (encoding) that allow more computations without data type overflow, and<br>
can allow the full plaintext polynomial to be utilized.

In [ ]:
def print_matrix(A, n):
    nrows = int(len(A)/n)
    for i in range(nrows):
        print("[", ", ".join(["{:.2f}".format(x) for x in A[n*i:(n*i+4)]]), ", ... ,",
              ", ".join(["{:.2f}".format(x) for x in A[(n*(i+1)-5):(n*(i+1))]]), "]")

In [ ]:
def print_vector(v):
    n = len(v)
    print("[", ", ".join(["{:.2f}".format(x) for x in v[0:4]]), ", ..., ",
          ", ".join(["{:.2f}".format(x) for x in v[(n-5):n]]))

In [ ]:
print("Example: Encoders / Integer Encoder")

IntegerEncoder] (For BFV scheme only)<br>
<br>
he IntegerEncoder encodes integers to BFV plaintext polynomials as follows.<br>
irst, a binary expansion of the integer is computed. Next, a polynomial is<br>
reated with the bits as coefficients. For example, the integer<br>
<br>
   26 = 2^4 + 2^3 + 2^1<br>
<br>
s encoded as the polynomial 1x^4 + 1x^3 + 1x^1. Conversely, plaintext<br>
olynomials are decoded by evaluating them at x=2. For negative numbers the<br>
ntegerEncoder simply stores all coefficients as either 0 or -1, where -1 is<br>
epresented by the unsigned integer plain_modulus - 1 in memory.<br>
<br>
ince encrypted computations operate on the polynomials rather than on the<br>
ncoded integers themselves, the polynomial coefficients will grow in the<br>
ourse of such computations. For example, computing the sum of the encrypted<br>
ncoded integer 26 with itself will result in an encrypted polynomial with<br>
arger coefficients: 2x^4 + 2x^3 + 2x^1. Squaring the encrypted encoded<br>
nteger 26 results also in increased coefficients due to cross-terms, namely,<br>
<br>
    (2x^4 + 2x^3 + 2x^1)^2 = 1x^8 + 2x^7 + 1x^6 + 2x^5 + 2x^4 + 1x^2;<br>
<br>
further computations will quickly increase the coefficients much more.<br>
Decoding will still work correctly in this case (evaluating the polynomial<br>
at x=2), but since the coefficients of plaintext polynomials are really<br>
integers modulo plain_modulus, implicit reduction modulo plain_modulus may<br>
yield unexpected results. For example, adding 1x^4 + 1x^3 + 1x^1 to itself<br>
plain_modulus many times will result in the constant polynomial 0, which is<br>
clearly not equal to 26 * plain_modulus. It can be difficult to predict when<br>
such overflow will take place especially when computing several sequential<br>
multiplications.<br>
<br>
The IntegerEncoder is easy to understand and use for simple computations,<br>
and can be a good tool to experiment with for users new to Microsoft SEAL.<br>
However, advanced users will probably prefer more efficient approaches,<br>
such as the BatchEncoder or the CKKSEncoder.

In [ ]:
parms = EncryptionParameters(scheme_type.BFV)
poly_modulus_degree = 4096
parms.set_poly_modulus_degree(poly_modulus_degree)
parms.set_coeff_modulus(CoeffModulus.BFVDefault(poly_modulus_degree))

There is no hidden logic behind our choice of the plain_modulus. The only<br>
thing that matters is that the plaintext polynomial coefficients will not<br>
exceed this value at any point during our computation; otherwise the result<br>
will be incorrect.

In [ ]:
parms.set_plain_modulus(512)
context = SEALContext.Create(parms)
print_parameters(context)

In [ ]:
keygen = KeyGenerator(context)
public_key = keygen.public_key()
secret_key = keygen.secret_key();
encryptor = Encryptor(context, public_key)
evaluator = Evaluator(context)
decryptor = Decryptor(context, secret_key)

We create an IntegerEncoder.

In [ ]:
encoder = IntegerEncoder(context)

First, we encode two integers as plaintext polynomials. Note that encoding<br>
is not encryption: at this point nothing is encrypted.

In [ ]:
value1 = 5
plain1 = encoder.encode(value1)
print("Encode {} as polynomial {} (plain1), ".format(value1, plain1.to_string()))

In [ ]:
value2 = -7
plain2 = encoder.encode(value2)
print("    encode {} as polynomial {} (plain2)".format(value2, plain2.to_string()))

Now we can encrypt the plaintext polynomials.

In [ ]:
encrypted1 = Ciphertext()
encrypted2 = Ciphertext()
print("Encrypt plain1 to encrypted1 and plain2 to encrypted2.")
encryptor.encrypt(plain1, encrypted1);
encryptor.encrypt(plain2, encrypted2);
print("    + Noise budget in encrypted1: {} bits".format(
        decryptor.invariant_noise_budget(encrypted1)))
print("    + Noise budget in encrypted2: {} bits".format(
        decryptor.invariant_noise_budget(encrypted2)))

As a simple example, we compute (-encrypted1 + encrypted2) * encrypted2.

In [ ]:
encryptor.encrypt(plain2, encrypted2)
encrypted_result = Ciphertext()
print("Compute encrypted_result = (-encrypted1 + encrypted2) * encrypted2.")
evaluator.negate(encrypted1, encrypted_result)
evaluator.add_inplace(encrypted_result, encrypted2)
evaluator.multiply_inplace(encrypted_result, encrypted2)
print("    + Noise budget in encrypted_result: {} bits".format(
        decryptor.invariant_noise_budget(encrypted_result)))
plain_result = Plaintext() 
print("Decrypt encrypted_result to plain_result.")
decryptor.decrypt(encrypted_result, plain_result);

Print the result plaintext polynomial. The coefficients are not even close<br>
to exceeding our plain_modulus, 512.

In [ ]:
print("    + Plaintext polynomial: {}".format(plain_result.to_string()))

Decode to obtain an integer result.

In [ ]:
print("Decode plain_result.")
print("    + Decoded integer: {} ...... Correct.".format(encoder.decode_int32(plain_result)))

########################################

In [ ]:
print("Example: Encoders / Batch Encoder");

[BatchEncoder] (For BFV scheme only)<br>
<br>
Let N denote the poly_modulus_degree and T denote the plain_modulus. Batching<br>
allows the BFV plaintext polynomials to be viewed as 2-by-(N/2) matrices, with<br>
each element an integer modulo T. In the matrix view, encrypted operations act<br>
element-wise on encrypted matrices, allowing the user to obtain speeds-ups of<br>
several orders of magnitude in fully vectorizable computations. Thus, in all<br>
but the simplest computations, batching should be the preferred method to use<br>
with BFV, and when used properly will result in implementations outperforming<br>
anything done with the IntegerEncoder.

In [ ]:
parms = EncryptionParameters(scheme_type.BFV)
poly_modulus_degree = 8192
parms.set_poly_modulus_degree(poly_modulus_degree)
parms.set_coeff_modulus(CoeffModulus.BFVDefault(poly_modulus_degree))

To enable batching, we need to set the plain_modulus to be a prime number<br>
congruent to 1 modulo 2*poly_modulus_degree. Microsoft SEAL provides a helper<br>
method for finding such a prime. In this example we create a 20-bit prime<br>
that supports batching.

In [ ]:
parms.set_plain_modulus(PlainModulus.Batching(poly_modulus_degree, 20))
context = SEALContext.Create(parms)
print_parameters(context)

We can verify that batching is indeed enabled by looking at the encryption<br>
parameter qualifiers created by SEALContext.<br>
#HERE

In [ ]:
qualifiers = context.qualifiers()
print("Batching enabled: {}".format(qualifiers.using_batching))

In [ ]:
keygen = KeyGenerator(context)
public_key = keygen.public_key()
secret_key = keygen.secret_key()
relin_keys = keygen.relin_keys()
encryptor = Encryptor(context, public_key)
evaluator = Evaluator(context)
decryptor = Decryptor(context, secret_key)

Batching is done through an instance of the BatchEncoder class.

In [ ]:
batch_encoder = BatchEncoder(context)

The total number of batching `slots' equals the poly_modulus_degree, N, and<br>
these slots are organized into 2-by-(N/2) matrices that can be encrypted and<br>
computed on. Each slot contains an integer modulo plain_modulus.

In [ ]:
slot_count = batch_encoder.slot_count()
row_size = int(slot_count / 2)
print("Plaintext matrix row size: {}".format(row_size))

The matrix plaintext is simply given to BatchEncoder as a flattened vector<br>
of numbers. The first `row_size' many numbers form the first row, and the<br>
rest form the second row. Here we create the following matrix:<br>
<br>
    [ 0,  1,  2,  3,  0,  0, ...,  0 ]<br>
    [ 4,  5,  6,  7,  0,  0, ...,  0 ]

In [ ]:
pod_matrix = Int64Vector([0] * slot_count)
pod_matrix[0] = 0
pod_matrix[1] = 1
pod_matrix[2] = 2
pod_matrix[3] = 3
pod_matrix[row_size] = 4
pod_matrix[row_size + 1] = 5
pod_matrix[row_size + 2] = 6
pod_matrix[row_size + 3] = 7

In [ ]:
print("Input plaintext matrix:")
print_matrix(pod_matrix, row_size)

First we use BatchEncoder to encode the matrix into a plaintext polynomial.

In [ ]:
plain_matrix = Plaintext() 
print("Encode plaintext matrix:")
batch_encoder.encode(pod_matrix, plain_matrix)

We can instantly decode to verify correctness of the encoding. Note that no<br>
encryption or decryption has yet taken place.

In [ ]:
print("    + Decode plaintext matrix ...... Correct.")
pod_result = Int64Vector([0] * slot_count)
batch_encoder.decode(plain_matrix, pod_result)
print_matrix(pod_result, row_size)

Next we encrypt the encoded plaintext.

In [ ]:
encrypted_matrix = Ciphertext()
print("Encrypt plain_matrix to encrypted_matrix.")
encryptor.encrypt(plain_matrix, encrypted_matrix)
print("    + Noise budget in encrypted_matrix: {} bits".format(
        decryptor.invariant_noise_budget(encrypted_matrix)))

Operating on the ciphertext results in homomorphic operations being performed<br>
simultaneously in all 8192 slots (matrix elements). To illustrate this, we<br>
form another plaintext matrix<br>
<br>
    [ 1,  2,  1,  2,  1,  2, ..., 2 ]<br>
    [ 1,  2,  1,  2,  1,  2, ..., 2 ]<br>
<br>
and encode it into a plaintext.

In [ ]:
pod_matrix2 = UInt64Vector([1,2]* row_size)
plain_matrix2 = Plaintext()
batch_encoder.encode(pod_matrix2, plain_matrix2);
print("Second input plaintext matrix:")
print_matrix(pod_matrix2, row_size);

We now add the second (plaintext) matrix to the encrypted matrix, and square<br>
the sum.

In [ ]:
print("Sum, square, and relinearize.")
evaluator.add_plain_inplace(encrypted_matrix, plain_matrix2)
evaluator.square_inplace(encrypted_matrix)
evaluator.relinearize_inplace(encrypted_matrix, relin_keys)

How much noise budget do we have left?

In [ ]:
print("    + Noise budget in result: {} bits".format(
        decryptor.invariant_noise_budget(encrypted_matrix)))

We decrypt and decompose the plaintext to recover the result as a matrix.

In [ ]:
plain_result = Plaintext()
print("Decrypt and decode result.")
decryptor.decrypt(encrypted_matrix, plain_result)
batch_encoder.decode(plain_result, pod_result)
print("    + Result plaintext matrix ...... Correct.")
print_matrix(pod_result, row_size)

Batching allows us to efficiently use the full plaintext polynomial when the<br>
desired encrypted computation is highly parallelizable. However, it has not<br>
solved the other problem mentioned in the beginning of this file: each slot<br>
holds only an integer modulo plain_modulus, and unless plain_modulus is very<br>
large, we can quickly encounter data type overflow and get unexpected results<br>
when integer computations are desired. Note that overflow cannot be detected<br>
in encrypted form. The CKKS scheme (and the CKKSEncoder) addresses the data<br>
type overflow issue, but at the cost of yielding only approximate results.

#############################################

In [ ]:
print("Example: Encoders / CKKS Encoder")

[CKKSEncoder] (For CKKS scheme only)<br>
<br>
In this example we demonstrate the Cheon-Kim-Kim-Song (CKKS) scheme for<br>
computing on encrypted real or complex numbers. We start by creating<br>
encryption parameters for the CKKS scheme. There are two important<br>
differences compared to the BFV scheme:<br>
<br>
    (1) CKKS does not use the plain_modulus encryption parameter;<br>
    (2) Selecting the coeff_modulus in a specific way can be very important<br>
        when using the CKKS scheme. We will explain this further in the file<br>
        `ckks_basics.cpp'. In this example we use CoeffModulus::Create to<br>
        generate 5 40-bit prime numbers.

In [ ]:
parms = EncryptionParameters(scheme_type.CKKS)

In [ ]:
poly_modulus_degree = 8192
parms.set_poly_modulus_degree(poly_modulus_degree)
parms.set_coeff_modulus(CoeffModulus.Create(
        poly_modulus_degree, IntVector([40, 40, 40, 40, 40])))

We create the SEALContext as usual and print the parameters.

In [ ]:
context = SEALContext.Create(parms)
print_parameters(context)

Keys are created the same way as for the BFV scheme.

In [ ]:
keygen = KeyGenerator(context)
public_key = keygen.public_key()
secret_key = keygen.secret_key()
relin_keys = keygen.relin_keys()

We also set up an Encryptor, Evaluator, and Decryptor as usual.

In [ ]:
encryptor = Encryptor(context, public_key)
evaluator = Evaluator(context)
decryptor = Decryptor(context, secret_key)

To create CKKS plaintexts we need a special encoder: there is no other way<br>
to create them. The IntegerEncoder and BatchEncoder cannot be used with the<br>
CKKS scheme. The CKKSEncoder encodes vectors of real or complex numbers into<br>
Plaintext objects, which can subsequently be encrypted. At a high level this<br>
looks a lot like what BatchEncoder does for the BFV scheme, but the theory<br>
behind it is completely different.

In [ ]:
    encoder = CKKSEncoder(context)

In CKKS the number of slots is poly_modulus_degree / 2 and each slot encodes<br>
one real or complex number. This should be contrasted with BatchEncoder in<br>
the BFV scheme, where the number of slots is equal to poly_modulus_degree<br>
and they are arranged into a matrix with two rows.

In [ ]:
slot_count = encoder.slot_count()
print("Number of slots: {}".format(slot_count))

We create a small vector to encode; the CKKSEncoder will implicitly pad it<br>
with zeros to full size (poly_modulus_degree / 2) when encoding.

In [ ]:
input = DoubleVector([ 0.0, 1.1, 2.2, 3.3 ])
print("Input vector: ")
print_vector(input);

Now we encode it with CKKSEncoder. The floating-point coefficients of `input'<br>
will be scaled up by the parameter `scale'. This is necessary since even in<br>
the CKKS scheme the plaintext elements are fundamentally polynomials with<br>
integer coefficients. It is instructive to think of the scale as determining<br>
the bit-precision of the encoding; naturally it will affect the precision of<br>
the result.<br>
<br>
In CKKS the message is stored modulo coeff_modulus (in BFV it is stored modulo<br>
plain_modulus), so the scaled message must not get too close to the total size<br>
of coeff_modulus. In this case our coeff_modulus is quite large (200 bits) so<br>
we have little to worry about in this regard. For this simple example a 30-bit<br>
scale is more than enough.

In [ ]:
plain = Plaintext() 
scale = 2.0**30
print("Encode input vector.")
encoder.encode(input, scale, plain)

We can instantly decode to check the correctness of encoding.

In [ ]:
output = DoubleVector()
print("    + Decode input vector ...... Correct.")
encoder.decode(plain, output)
print_vector(output)

The vector is encrypted the same was as in BFV.

In [ ]:
encrypted = Ciphertext() 
print("Encrypt input vector, square, and relinearize.")
encryptor.encrypt(plain, encrypted)

Basic operations on the ciphertexts are still easy to do. Here we square the<br>
ciphertext, decrypt, decode, and print the result. We note also that decoding<br>
returns a vector of full size (poly_modulus_degree / 2); this is because of<br>
the implicit zero-padding mentioned above.

In [ ]:
evaluator.square_inplace(encrypted)
evaluator.relinearize_inplace(encrypted, relin_keys)

We notice that the scale in the result has increased. In fact, it is now the<br>
square of the original scale: 2^60.

In [ ]:
print("    + Scale in squared input: {} ( {} bits)".format(
        encrypted.scale(), log2(encrypted.scale()))) 

In [ ]:
print("Decrypt and decode.")
decryptor.decrypt(encrypted, plain)
encoder.decode(plain, output)
print("    + Result vector ...... Correct.")
print_vector(output)

The CKKS scheme allows the scale to be reduced between encrypted computations.<br>
This is a fundamental and critical feature that makes CKKS very powerful and<br>
flexible. We will discuss it in great detail in `3_levels.cpp' and later in<br>
`4_ckks_basics.cpp'.